In [1]:
# Importing libraries
from keras.utils.np_utils import to_categorical

import pandas as pd
import numpy as np

import re

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from itertools import combinations
from sklearn.cluster import KMeans

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import tensorflow as tf
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub
import numpy as np

import matplotlib.pyplot as plt
import nltk

nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('stopwords')

nltk.download('punkt')

import spacy 
from spacy import displacy

import random

# from rake_nltk import Rake

from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Embedding,Dense

import os
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorboard.plugins import projector

from gensim.scripts.glove2word2vec import glove2word2vec

# import en_core_web_lg
# nlp = en_core_web_lg.load()

from nltk.tokenize import word_tokenize

import gensim
from scipy import spatial

import csv
import tensorflow as tf

# import transformers
# transformers.logging.set_verbosity(transformers.logging.CRITICAL)

# from sentence_transformers import SentenceTransformer, util
# import nmslib

# distilbert = SentenceTransformer('distilbert-base-uncased')

import multiprocessing
import math

import time
from gensim.models import KeyedVectors
from gensim.models.fasttext import FastText
import pandas as pd
import gensim
gensim.__version__

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


'3.6.0'

In [2]:
#Importing dataset:
data = pd.read_csv("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Capstone Project/ml-25m/movie_all_info.csv")
movtag4 = pd.read_pickle("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Capstone Project/Final datasets/2_ner_tags.pkl")

In [3]:
movtag4.head()

,movieId,combined_tags_nodup,tag_people,tag_org,tag_norp,tag_location,tags
0,1,"[Owned, imdb, top, Pixar, time, travel, childr...","[chindren pixar, Tom Hanks, Woody, Tim Allen, ...","[imdb, Pixar, Disney, National Film Registry, ...",[],[],"[top, time, travel, comedy, funny, witty, anim..."
1,2,"[Robin, Williams, time, travel, fantasy, based...","[Robin Williams, Chris Van Allsburg, Kirsten D...",[Zathura],[],[],"[Robin, time, travel, fantasy, based, book, bo..."
2,3,"[funny, best, friend, duringcreditsstinger, fi...","[Jack Lemmon, Walter Matthau grun, Howard Deut...",[comedinha de velhinhos engraÃƒÂ§ada old],[],[],"[funny, best, friend, fishing, old, man, seque..."
3,4,"[based, novel, book, chick, flick, divorce, in...",[CLV],[],[],[],"[based, novel, book, chick, flick, divorce, in..."
4,5,"[aging, baby, confidence, contraception, daugh...","[steve martin Comedy Touching CLV, Diane Keato...",[],[],[],"[aging, baby, confidence, contraception, daugh..."


In [8]:
#creating a copy of the dataframe

def keras_embedding(dataset, ratings_dataset, colname):
#joining elements in all lists
  movtag_taglst = [' '.join(d) for d in dataset[colname]]

  Vocab_size = 100000
  encoded_reviews = [one_hot(d,Vocab_size) for d in movtag_taglst]

  max_length = 1000 #taking only 1000 words from tags for each movie.
  padded_reviews = pad_sequences(encoded_reviews,maxlen=max_length,padding='post')

  #y-labels for training the neural network
  labels = ratings_dataset['averageRating_imdb']
  labels = np.array(labels)
  labels[np.where(np.isnan(labels))] = 0
  labels = to_categorical(labels)

  #Making the NN architecture
  model = Sequential()
  embedding_layer = Embedding(input_dim=Vocab_size,output_dim=40,input_length=max_length)
  model.add(embedding_layer)
  model.add(Flatten())
  model.add(Dense(10,activation='sigmoid'))
  model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
  print(model.summary())

  #Fitting the model
  model.fit(padded_reviews,labels,epochs=10)

  #appending the word embeddings in the dataframe
  #To create one embedding for all tags for a movie, we average out the individual word embeddings. This is because, the order of the tags
  #does not matter here.
  embeddings = []
  for i in padded_reviews:
    lst = np.array(embedding_layer.get_weights()[0][i])
    embeddings.append(np.sum(lst, axis = 0)/len(lst))

  dataset['keras'+colname] = embeddings

In [9]:
#generating keras embeddings for tag people
%%time
keras_embedding(movtag4, data, 'tag_people')

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1000, 40)          4000000   
                                                                 
 flatten_2 (Flatten)         (None, 40000)             0         
                                                                 
 dense_2 (Dense)             (None, 10)                400010    
                                                                 
Total params: 4,400,010
Trainable params: 4,400,010
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
1951/1951 [==============================] - 32s 16ms/step - loss: 1.5522 - acc: 0.3817
Epoch 2/10
1951/1951 [==============================] - 16s 8ms/step - loss: 1.4715 - acc: 0.4224
Epoch 3/10
1951/1951 [==============================] - 18s 9ms/step - loss: 1.3970 - acc: 0.4579
Epoch 4/

In [11]:
#generating keras embeddings for tag org
%%time
keras_embedding(movtag4, data, 'tag_org')

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 1000, 40)          4000000   
                                                                 
 flatten_4 (Flatten)         (None, 40000)             0         
                                                                 
 dense_4 (Dense)             (None, 10)                400010    
                                                                 
Total params: 4,400,010
Trainable params: 4,400,010
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
1951/1951 [==============================] - 35s 17ms/step - loss: 1.5713 - acc: 0.3772
Epoch 2/10
1951/1951 [==============================] - 21s 11ms/step - loss: 1.5351 - acc: 0.3918
Epoch 3/10
1951/1951 [==============================] - 21s 11ms/step - loss: 1.5055 - acc: 0.4038
Epoch 

In [12]:
#generating keras embeddings for tag norp
%%time
keras_embedding(movtag4, data, 'tag_norp')

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 1000, 40)          4000000   
                                                                 
 flatten_5 (Flatten)         (None, 40000)             0         
                                                                 
 dense_5 (Dense)             (None, 10)                400010    
                                                                 
Total params: 4,400,010
Trainable params: 4,400,010
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
1951/1951 [==============================] - 30s 15ms/step - loss: 1.5848 - acc: 0.3695
Epoch 2/10
1951/1951 [==============================] - 41s 21ms/step - loss: 1.5760 - acc: 0.3713
Epoch 3/10
1951/1951 [==============================] - 23s 12ms/step - loss: 1.5687 - acc: 0.3760
Epoch 

In [10]:
#generating keras embeddings for tag location
%%time
keras_embedding(movtag4, data, 'tag_location')

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 1000, 40)          4000000   
                                                                 
 flatten_3 (Flatten)         (None, 40000)             0         
                                                                 
 dense_3 (Dense)             (None, 10)                400010    
                                                                 
Total params: 4,400,010
Trainable params: 4,400,010
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
1951/1951 [==============================] - 28s 14ms/step - loss: 1.5835 - acc: 0.3697
Epoch 2/10
1951/1951 [==============================] - 22s 11ms/step - loss: 1.5671 - acc: 0.3776
Epoch 3/10
1951/1951 [==============================] - 19s 10ms/step - loss: 1.5548 - acc: 0.3833
Epoch 

In [13]:
#generating keras embeddings for tag location
%%time
keras_embedding(movtag4, data, 'tags')

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 1000, 40)          4000000   
                                                                 
 flatten_6 (Flatten)         (None, 40000)             0         
                                                                 
 dense_6 (Dense)             (None, 10)                400010    
                                                                 
Total params: 4,400,010
Trainable params: 4,400,010
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
1951/1951 [==============================] - 41s 20ms/step - loss: 1.4472 - acc: 0.4145
Epoch 2/10
1951/1951 [==============================] - 48s 25ms/step - loss: 1.3220 - acc: 0.4661
Epoch 3/10
1951/1951 [==============================] - 43s 22ms/step - loss: 1.2130 - acc: 0.5237
Epoch 

In [15]:
movtag4.to_pickle("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Capstone Project/Final datasets/4_Keras_tags.pkl")